<a href="https://colab.research.google.com/github/mreisenauer/Emerging-Tech-in-Water-Sustainability-Project/blob/main/Analyzing_Evaporation_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Authenticate & Select Data

In [ ]:
# Clear all variables
%reset -f

# Specify which sheet you want to work with:
title="Stalk 20"

# Authenticate Google User
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)



## Define Evaporation Area


### 2D (Circular)

In [ ]:
# Initialize
import math as m

# Area of Circle
d_circle_cm = 7 #Units: cm
A_circle_cm2 = m.pi*(d_circle_cm/2)**2 #Units: cm2

# Area Calculations
A_evap_m2 = A_circle_cm2/10000 #Units: m2
A_proj_m2 = A_evap_m2 #Units: m2
EAI = A_evap_m2/A_proj_m2
EAI

### 3D (Single Cylinder)

In [ ]:
# Initialize
import math as m

# Area of Cylinder
d_cylinder_cm = 0.75 #Units: cm
h_cylinder_cm = 8.11 #Units: cm
SA_cylinder_cm2=(m.pi*(d_cylinder_cm/2)**2)+(m.pi*d_cylinder_cm*h_cylinder_cm) #Units: cm2

# Area Calculations
A_evap_m2 = SA_cylinder_cm2/10000
A_proj_m2 = (m.pi*(d_cylinder_cm/2)**2)/10000
EAI = A_evap_m2/A_proj_m2
print(SA_cylinder_cm2,
A_evap_m2,
A_proj_m2,EAI)

### 3D (Multiple Cylinders + Circular Base)

In [ ]:
# Initialize
import math as m

# Area Contributions by Cylinder(s)
n_cylinder=7
d_cylinder_cm = 0.75 #Units: cm
h_cylinder_cm = 8.05 #Units: cm
SA_cylinder_cm2=n_cylinder*((m.pi*(d_cylinder_cm/2)**2)+(m.pi*d_cylinder_cm*h_cylinder_cm)) #Units: cm2 / Notes: SA_cylinder_cm2 does not include bottom area.

# Area Contributions by Base
d_base_cm = 8.51 #Units: cm
SA_base_cm2 = (m.pi*(d_base_cm/2)**2)-(n_cylinder*(m.pi*(d_cylinder_cm/2)**2)) #Units: cm2


# Area Calculations
A_evap_m2 = (SA_cylinder_cm2)/10000
A_proj_m2 = (m.pi*(d_base_cm/2)**2)/10000
EAI = A_evap_m2/A_proj_m2



## 3D (Total Evaporation Surface Area: Sticks Only)

In [ ]:
# Initialize
import math as m

# Area Contributions by Cylinder(s)
n_cylinder=1
d_cylinder_cm = 0.75 #Units: cm
h_cylinder_cm = 11.1 #Units: cm
SA_cylinder_cm2=n_cylinder*((m.pi*(d_cylinder_cm/2)**2)+(m.pi*d_cylinder_cm*h_cylinder_cm)) #Units: cm2 / Notes: SA_cylinder_cm2 does not include bottom area.


# Area Calculations
d_base_cm = 7 #cm
A_evap_m2 = (SA_cylinder_cm2)/10000
A_proj_m2 = (m.pi*(d_base_cm/2)**2)/10000

EAI = 1 + (4*h_cylinder_cm)/d_cylinder_cm
print(EAI)

### 3D (Multiple Cylinders + Hexagonal Base)

In [ ]:
# Initialize
import math as m

# Area Contributions by Cylinder(s)
n_cylinder=7
d_cylinder_cm = 0.75 #Units: cm
h_cylinder_cm = 7.5 #Units: cm
SA_cylinder_cm2=n_cylinder*((m.pi*(d_cylinder_cm/2)**2)+(m.pi*d_cylinder_cm*h_cylinder_cm)) #Units: cm2 / Notes: SA_cylinder_cm2 does not include bottom area.

# Area Contributions by Base
s_base_cm = 4 #Units: cm distance between sticks
SA_base_cm2 = (3*m.sqrt(3)*(s_base_cm**2)/2)-(n_cylinder*(m.pi*(d_cylinder_cm/2)**2)) #Units: cm2


# Area Calculations
A_evap_m2 = ((2/7)*SA_cylinder_cm2)/10000
A_proj_m2 = (3*m.sqrt(3)*(s_base_cm**2)/2)/10000
EAI = A_evap_m2/A_proj_m2

In [ ]:
EAI


## Import Data

In [ ]:
# Import data from google sheets
import pandas as pd
import numpy as np

wb = gc.open(title)

# Specify which tab in the data collection spreadsheet:
tab1 = wb.worksheet('Sheet1')

# Extract data from the specified tab:
data_raw = tab1.get_all_values()

# Get basic info about imported data:
df_raw = pd.DataFrame(data_raw)
#df_raw.columns = df_raw.iloc[0] #Renames columns based on values in top row.
#df_raw = df_raw.iloc[1:] #Redefines df_raw to exclude the top row (which is now column names).


# Extract processing aspects from original dataframe
run_column=[]
run_length = []
run_length_env = []
run_name = []
for i_run in range(1,df_raw.shape[1],4):
  run_column.append(i_run)
  run_name.append(df_raw[i_run][0])
  df_raw[i_run] = df_raw[i_run].replace('',np.nan) # Replaces blank spaces ('') with NaN
  df_raw[i_run+1] = df_raw[i_run+1].replace('',np.nan) #Does the same as the line before, but for env data
  run_length.append(np.sum(df_raw[i_run].count())-1) # Counts length of each run array (excluding Row 0, which is the column name)
  run_length_env.append(np.sum(df_raw[i_run+1].count())-1)

# Make a dataframe for Evaporation Data
df_evap_raw = df_raw[1:][run_column].astype(float).reset_index(drop=True)
df_evap_raw.columns = range(df_evap_raw.shape[1])

# Make a dataframe for Environmental Data
run_column_env=list(np.array(run_column)+1)
df_env_raw = df_raw[1:][run_column_env].reset_index(drop=True)
df_env_raw.columns = range(df_evap_raw.shape[1])

# Split Environmental Data into its component parts
H1=[None]*df_env_raw.shape[1]
T1=[None]*df_env_raw.shape[1]
H2=[None]*df_env_raw.shape[1]
T2=[None]*df_env_raw.shape[1]
T3=[None]*df_env_raw.shape[1]

for i_run_env in range(df_env_raw.shape[1]):
  H1[i_run_env]=df_env_raw[i_run_env][0:run_length_env[i_run_env]].str.split(' , ', expand=True)[0].astype(float)
  T1[i_run_env]=df_env_raw[i_run_env][0:run_length_env[i_run_env]].str.split(' , ', expand=True)[1].astype(float)
  H2[i_run_env]=df_env_raw[i_run_env][0:run_length_env[i_run_env]].str.split(' , ', expand=True)[2].astype(float)
  T2[i_run_env]=df_env_raw[i_run_env][0:run_length_env[i_run_env]].str.split(' , ', expand=True)[3].astype(float)
  T3[i_run_env]=df_env_raw[i_run_env][0:run_length_env[i_run_env]].str.split(' , ', expand=True)[4].astype(float)

  # Conform to evaporation data lengths.
H1_corrected=[None]*df_env_raw.shape[1]
T1_corrected=[None]*df_env_raw.shape[1]
H2_corrected=[None]*df_env_raw.shape[1]
T2_corrected=[None]*df_env_raw.shape[1]
T3_corrected=[None]*df_env_raw.shape[1]

for i_run_env2 in range(df_env_raw.shape[1]):
  if run_length[i_run_env2] > run_length_env[i_run_env2]:
    t_env=np.linspace(0,run_length[i_run_env2],run_length_env[i_run_env2])
    H1_corrected[i_run_env2]=np.interp(list(range(run_length[i_run_env2])),t_env,H1[i_run_env2])
    T1_corrected[i_run_env2]=np.interp(list(range(run_length[i_run_env2])),t_env,T1[i_run_env2])
    H2_corrected[i_run_env2]=np.interp(list(range(run_length[i_run_env2])),t_env,H2[i_run_env2])
    T2_corrected[i_run_env2]=np.interp(list(range(run_length[i_run_env2])),t_env,T2[i_run_env2])
    T3_corrected[i_run_env2]=np.interp(list(range(run_length[i_run_env2])),t_env,T3[i_run_env2])
  else:
    H1_corrected[i_run_env2]=H1[i_run_env2][0:run_length[i_run_env2]]
    T1_corrected[i_run_env2]=T1[i_run_env2][0:run_length[i_run_env2]]
    H2_corrected[i_run_env2]=H2[i_run_env2][0:run_length[i_run_env2]]
    T2_corrected[i_run_env2]=T2[i_run_env2][0:run_length[i_run_env2]]
    T3_corrected[i_run_env2]=T3[i_run_env2][0:run_length[i_run_env2]]


## Clean Data

In [ ]:
# Make copies so dataframes/arrays can be modified without losing original data
df_evap1=df_evap_raw.copy()
run_length1=run_length.copy()


# Converting from "mass[g] vs t[min]" to "mass evaporated[kg/m2] vs. t[min]"
for i_run1 in range(df_evap1.shape[1]):
  for i_length1 in range(run_length1[i_run1]):
    df_evap1[i_run1][i_length1] = ((df_evap_raw[i_run1][0]-df_evap_raw[i_run1][i_length1])/1000/A_evap_m2)*EAI


# Clean data using plotly and ipywidgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px

x_array1=[None]*df_evap1.shape[1]
y_array1=[None]*df_evap1.shape[1]
evap_original=[None]*df_evap1.shape[1]

for i_clean1 in range(df_evap1.shape[1]):
  x_array1[i_clean1]=range(run_length1[i_clean1])
  y_array1[i_clean1]=df_evap1[i_clean1][0:run_length1[i_clean1]]
  evap_original[i_clean1]=go.Scatter(x=list(x_array1[i_clean1]),y=y_array1[i_clean1])


field_start = [None]*len(run_name)
field_end = [None]*len(run_name)
tab_contents = field_start

out = [widgets.Output() for i_name1 in range(len(run_name))]

tab = widgets.Tab(out)

for i_name2 in range(len(run_name)):

    with out[i_name2]:
       fig = go.FigureWidget(evap_original[i_name2])
       fig.update_layout(height=300,width=800,
                         xaxis_title="Time [min]",
                         yaxis_title="Evaporation [kg m-2]",
                         margin=dict(l=0,r=0,t=0,b=0))
       fig.show()
       field_start[i_name2] = widgets.BoundedIntText(value=0,min=0,max=run_length1[i_name2],step=1,description="Start: ")
       display(field_start[i_name2])
       field_end[i_name2] = widgets.BoundedIntText(value=run_length1[i_name2],min=0,max=run_length1[i_name2],step=1,description="End: ")
       display(field_end[i_name2])
    tab.set_title(i_name2, run_name[i_name2])

    with out[i_name2]:
       fig2 = make_subplots(specs=[[{"secondary_y": True}]])
       fig2.add_trace(go.Scatter(x=list(x_array1[i_name2]),y=H1_corrected[i_name2],name="H1",line=dict(color='rgb(117,112,179)'),mode='lines'),secondary_y=False)
       fig2.add_trace(go.Scatter(x=list(x_array1[i_name2]),y=H2_corrected[i_name2],name="H2",line=dict(color='rgb(141,160,203)'),mode='lines'),secondary_y=False)
       fig2.add_trace(go.Scatter(x=list(x_array1[i_name2]),y=T1_corrected[i_name2],name="T1",line=dict(color='rgb(217,95,2)'),mode='lines'),secondary_y=True)
       fig2.add_trace(go.Scatter(x=list(x_array1[i_name2]),y=T2_corrected[i_name2],name="T2",line=dict(color='rgb(252,141,98)'),mode='lines'),secondary_y=True)
       fig2.add_trace(go.Scatter(x=list(x_array1[i_name2]),y=T3_corrected[i_name2],name="T3",line=dict(color='rgb(253,205,172)'),mode='lines'),secondary_y=True)
       fig2.update_layout(height=300,width=850,
                         xaxis_title="Time [min]",
                         margin=dict(l=0,r=0,t=0,b=0))
       fig2.update_yaxes(title_text="Relative Humidity [%]",title_font_color='rgb(117,112,179)', secondary_y=False)
       fig2.update_yaxes(title_text="Temperature [C]",title_font_color='rgb(217,95,2)', secondary_y=True)
       fig2.update_yaxes(range=[0, 80], secondary_y=False)
       fig2.update_yaxes(range=[15, 40], secondary_y=True)
       fig2.update_layout(legend=dict(orientation='h',yanchor='bottom',y=1.03,xanchor='right',x=0.94))
       fig2.show()

display(tab)


In [ ]:
x_array2=[None]*df_evap1.shape[1]
y_array2=[None]*df_evap1.shape[1]
x_length=[None]*df_evap1.shape[1]
evap_clean=[None]*df_evap1.shape[1]

H1_clean=[None]*df_evap1.shape[1]
H2_clean=[None]*df_evap1.shape[1]
T1_clean=[None]*df_evap1.shape[1]
T2_clean=[None]*df_evap1.shape[1]
T3_clean=[None]*df_evap1.shape[1]

for i_clean2 in range(df_evap1.shape[1]):
  x_array2[i_clean2]=range(int(field_end[i_clean2].value)-int(field_start[i_clean2].value))
  x_length[i_clean2]=int(field_end[i_clean2].value)-int(field_start[i_clean2].value)
  y_array2[i_clean2]=df_evap1[i_clean2][int(field_start[i_clean2].value):int(field_end[i_clean2].value)]-df_evap1[i_clean2][int(field_start[i_clean2].value)]
  evap_clean[i_clean2]=go.Scatter(x=list(x_array2[i_clean2]),y=y_array2[i_clean2],name=run_name[i_clean2])
  H1_clean[i_clean2]=H1_corrected[i_clean2][int(field_start[i_clean2].value):int(field_end[i_clean2].value)]
  H2_clean[i_clean2]=H2_corrected[i_clean2][int(field_start[i_clean2].value):int(field_end[i_clean2].value)]
  T1_clean[i_clean2]=T1_corrected[i_clean2][int(field_start[i_clean2].value):int(field_end[i_clean2].value)]
  T2_clean[i_clean2]=T2_corrected[i_clean2][int(field_start[i_clean2].value):int(field_end[i_clean2].value)]
  T3_clean[i_clean2]=T3_corrected[i_clean2][int(field_start[i_clean2].value):int(field_end[i_clean2].value)]

fig_clean = go.Figure(evap_clean)
fig_clean.update_layout(height=500,width=800,
                         xaxis_title="Time [min]",
                         yaxis_title="Evaporation [kg m-2]",
                         margin=dict(l=0,r=0,t=30,b=0))
fig_clean.show()

# Table displaying environmental conditions
T_table=[]
H_table=[]
for i_table in range(df_env_raw.shape[1]):
  T_table.extend([np.round(np.average([T1_clean[i_table],T2_clean[i_table]]),2)])
  H_table.extend([np.round(np.average([H1_clean[i_table],H2_clean[i_table]]),2)])

fig_table = go.Figure(data=[go.Table(header=dict(values=['Run','Avg Humidity [%]','Avg Temperature [C]']),
                 cells=dict(values=[run_name, H_table, T_table]))
                     ])
fig_table.update_layout(height=150, width=700, margin=dict(l=50,r=0,t=20,b=0))

fig_table.show()


In [ ]:
print(T_table)

## Process & Export Data

In [ ]:
m_evap2=[None]*df_evap1.shape[1]
b_evap2=[None]*df_evap1.shape[1]
for i_process in range(df_evap1.shape[1]):
  m_evap2[i_process]=np.polyfit(x_array2[i_process],y_array2[i_process],1)[0]
  b_evap2[i_process]=np.polyfit(x_array2[i_process],y_array2[i_process],1)[1]

flux_LMH=np.array(m_evap2)*60


# Information for additional columns (y-linear and Flux/m/b)
y_linear=[None]*df_evap1.shape[1]
data_processed=[None]*df_evap1.shape[1]
for i_process2 in range(df_evap1.shape[1]):
  y_linear[i_process2]=m_evap2[i_process2]*np.array(x_array2[i_process2])+b_evap2[i_process2]
  data_processed[i_process2]=["Flux [LMH]: " + run_name[i_process2],flux_LMH[i_process2],"m (linear)",m_evap2[i_process],"b (linear)",b_evap2[i_process]]

# Combine into a single dataframe
combined_data=[pd.Series(range(max(x_length)),name="Time [min]")]
for i_combine in range(df_evap1.shape[1]):
  combined_data.extend([y_array2[i_combine].rename(run_name[i_combine]+" [kg m-2]"),pd.Series(y_linear[i_combine],name=run_name[i_combine]+": Linear Fit"),pd.Series(data_processed[i_combine],name=run_name[i_combine]+" Info"),pd.Series("",name="")])

df_combined=pd.DataFrame(combined_data)
df_evap2=df_combined.transpose().apply(lambda x: pd.Series(x.dropna().values)).fillna('') #Resets the index value/numbering for each run.

# Make an environmental conditions dataframe
env_data=[pd.Series(range(max(x_length)),name="Time [min]")]
for i_env in range(df_evap1.shape[1]):
  env_data.extend([pd.Series(np.round(H1_clean[i_env],2),name="H1: "+run_name[i_combine]),pd.Series(np.round(H2_clean[i_env],2),name="H2: "+run_name[i_combine]),
                  pd.Series(np.round(T1_clean[i_env],2),name="T1: "+run_name[i_combine]),pd.Series(np.round(T2_clean[i_env],2),name="T2: "+run_name[i_combine]),pd.Series(np.round(T3_clean[i_env],2),name="T3: "+run_name[i_combine]),
                  pd.Series("",name="")])

df_env_temp=pd.DataFrame(env_data)
df_env2=df_env_temp.transpose().apply(lambda x: pd.Series(x.dropna().values)).fillna('') #Resets the index value/numbering for each run.


In [ ]:
# authenticate
#from google.colab import auth
#auth.authenticate_user()
#import gspread
#from oauth2client.client import GoogleCredentials as GC
#gc = gspread.authorize(GC.get_application_default())

# Authenticate Google User
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Install to allow to export/format data
!pip install gspread-formatting

# For Evaporation Data
# Open sheets and creating worksheet
sh = gc.open(title)
ws = sh.add_worksheet(title="Processed", rows=str(df_evap2.shape[0]), cols=str(df_evap2.shape[1]))

# Export dataframe to "Processed" worksheet
from gspread_dataframe import set_with_dataframe
set_with_dataframe(ws, df_evap2)

# Format top row to be in bold format
from gspread_formatting import *
fmt_bold = cellFormat(
    textFormat=textFormat(bold=True)
    )
format_cell_range(ws, '1:1', fmt_bold)

# For Environmental Data
# Open sheets and creating worksheet
ws2 = sh.add_worksheet(title="Env Data", rows=str(df_env2.shape[0]), cols=str(df_env2.shape[1]))

# Export dataframe to "Env Data" worksheet
from gspread_dataframe import set_with_dataframe
set_with_dataframe(ws2, df_env2)

# Format top row to be in bold format
from gspread_formatting import *
fmt_bold = cellFormat(
    textFormat=textFormat(bold=True)
    )
format_cell_range(ws2, '1:1', fmt_bold)